# Model sec2sec z użyciem wyrazów

## Stałe

In [32]:
batch_size = 64  
epochs = 1  
latent_dim = 256  
embed_dim = 32
num_samples = 10000
data_path = './data/pol-eng-short.txt'
clean_data_path = './data/clean-pol-eng-short.txt'

## Czyszczenie tekstu

In [3]:
import string

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read()
    table = str.maketrans('', '', string.punctuation)
    clean_lines = lines.lower().translate(table)

with open(clean_data_path, 'w', encoding='utf-8') as f:
    f.write(clean_lines)

## Wektoryzacja

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()

with open(clean_data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: num_samples]:
    input_text, target_text = line.split('\t')
    #input_text = '<s> ' + input_text + ' <e>'
    target_text = "<s> " + target_text + " <e>"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for word in input_text.split(' '):
        if word not in input_words:
            input_words.add(word)
    for word in target_text.split(' '):
        if word not in target_words:
            target_words.add(word)
            
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt.split(' ')) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split(' ')) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 3221
Number of unique output tokens: 6246
Max sequence length for inputs: 7
Max sequence length for outputs: 11


## Generacja danych

In [5]:
import numpy as np

# slowniki slow
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

# wypelnienie wejsc zerami
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for text_index, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for word_index, (word) in enumerate(input_text.split(' ')):
        encoder_input_data[text_index, word_index] = input_token_index[word]
    for word_index, (word) in enumerate(target_text.split(' ')):
        decoder_input_data[text_index, word_index] = target_token_index[word]
        if word_index > 0:
            decoder_target_data[text_index, word_index-1, target_token_index[word]] = 1.

## Model treningowy

In [33]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_embedding)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(decoder_lstm)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model\
model.save('s2s-word-embedding.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 69s 9ms/step - loss: 2.0062 - val_loss: 2.5572


/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_46 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_45/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_45/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


## Model produkcyjny

In [38]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense

model = load_model('s2s-word-embedding.h5')

for layer in model.layers:
    print(layer.get_config())
    print('\n')
    
#inputs
encoder_inputs = Input(shape=(None,))
decoder_inputs = Input(shape=(None,))

#inference encoder
encoder_embedding = model.get_layer(index = 2)(encoder_inputs)
encoder_lstm = model.get_layer(index = 4)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

#inference decoder
decoder_embedding = model.get_layer(index = 3)(decoder_inputs)
decoder_lstm = model.get_layer(index = 5)
decoder_dense = model.get_layer(index = 6)
    
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
#reverse_input_char_index = dict(
#    (i, char) for char, i in input_token_index.items())
reverse_input_word_index = dict(
    (i, word) for word, i in input_token_index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in target_token_index.items())



def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["<s>"]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += sampled_word + " "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == "<e>" or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

{'name': 'input_1', 'dtype': 'float32', 'batch_input_shape': (None, None), 'sparse': False}


{'name': 'input_2', 'dtype': 'float32', 'batch_input_shape': (None, None), 'sparse': False}


{'trainable': True, 'mask_zero': False, 'name': 'embedding_1', 'embeddings_initializer': {'config': {'minval': -0.05, 'seed': None, 'maxval': 0.05}, 'class_name': 'RandomUniform'}, 'activity_regularizer': None, 'embeddings_regularizer': None, 'input_length': None, 'output_dim': 256, 'input_dim': 3221, 'embeddings_constraint': None, 'batch_input_shape': (None, None), 'dtype': 'float32'}


{'trainable': True, 'mask_zero': False, 'name': 'embedding_2', 'embeddings_initializer': {'config': {'minval': -0.05, 'seed': None, 'maxval': 0.05}, 'class_name': 'RandomUniform'}, 'activity_regularizer': None, 'embeddings_regularizer': None, 'input_length': None, 'output_dim': 256, 'input_dim': 6246, 'embeddings_constraint': None, 'batch_input_shape': (None, None), 'dtype': 'float32'}


{'trainable': True, 'kernel_in

-
Input sentence: welcome
Decoded sentence: na mnie <e> 
-
Input sentence: welcome
Decoded sentence: na mnie <e> 
-
Input sentence: who ate
Decoded sentence: kto zostaje 
-
Input sentence: who won
Decoded sentence: kto wygrał <e> 
-
Input sentence: you run
Decoded sentence: ty lekarzem 
-
Input sentence: you won
Decoded sentence: wygrałeś <e> 
-
Input sentence: back off
Decoded sentence: odsuń się <e> 
-
Input sentence: back off
Decoded sentence: odsuń się <e> 
